# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118960e+02     1.557452e+00
 * time: 0.8576209545135498
     1     1.095154e+01     9.715449e-01
 * time: 3.5115599632263184
     2    -1.145865e+01     1.057389e+00
 * time: 4.228432893753052
     3    -3.410529e+01     7.985129e-01
 * time: 5.326939821243286
     4    -4.761595e+01     5.779691e-01
 * time: 6.385800838470459
     5    -5.700069e+01     2.074320e-01
 * time: 7.472216844558716
     6    -5.986647e+01     1.279507e-01
 * time: 8.188549041748047
     7    -6.088289e+01     4.977011e-02
 * time: 8.903663873672485
     8    -6.125159e+01     8.334156e-02
 * time: 9.61521601676941
     9    -6.156794e+01     3.637489e-02
 * time: 10.313954830169678
    10    -6.177810e+01     3.313609e-02
 * time: 11.011951923370361
    11    -6.195129e+01     2.489472e-02
 * time: 11.711100816726685
    12    -6.202658e+01     2.452130e-02
 * time: 12.392146825790405
    13    -6.210222e+01     2.109270e-02
 * time: 13.082871913909912
 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557679
    AtomicNonlocal      14.8522642
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336822

    total               -62.261666460805
